## Section 1: Data Importing

In [ ]:
#Importing packages for data cleaning and manipulation
import sys
sys.path.append('../') #Ensures that the parent directory is in Python's search path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
#Importing data obtained from "https://archive.ics.uci.edu/dataset/73/mushroom"
data_path = os.path.join('..', 'data', 'agaricus-lepiota.data')
column_names = [
    'class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
    'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape',
    'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',
    'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color',
    'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat'
]
lepiota_data = pd.read_csv(data_path, header=None, names=column_names)

In [ ]:
lepiota_data.head()

## Section 2: Data Cleansing

In [ ]:

missing = pd.concat([lepiota_data.isnull().sum(), 100 * lepiota_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')